In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [65]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [66]:
print(f'length of the dataset: {len(text)}')
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 30000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200

length of the dataset: 1115394


In [67]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars) ,vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [68]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [69]:
# let's encode our entire dataset and create a tensor from the encoder
data = torch.tensor(encode(text), dtype=torch.long)
data.dtype, data.shape

(torch.int64, torch.Size([1115394]))

In [70]:
# let's separate the data into training and validation sets
n = int(0.9 * len(data))
training = data[:n]  # 90%
val = data[n:]       # 10%

In [71]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = training if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[1+i:i+block_size+1] for i in ix])
    return x, y

@torch.no_grad()
def estimateloss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

xb, yb = get_batch('train')  # getting tensor of shape (4,8), in better words, getting random mini-batches of shape (4,8)

In [72]:
torch.manual_seed(1337)
class BiagramModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_tabe = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor integers
        logits = self.token_embedding_tabe(idx)  # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sample index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
model = BiagramModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)
print(decode(model.generate(torch.zeros(1,1, dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.7313, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [73]:
# create a torch.optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [74]:
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % 1000 == 0:
        losses = estimateloss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    # sample a batch from the data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.7312, val loss 4.7249
step 1000: train loss 2.4895, val loss 2.5200
step 2000: train loss 2.4723, val loss 2.4919
step 3000: train loss 2.4616, val loss 2.4859
step 4000: train loss 2.4597, val loss 2.4847
step 5000: train loss 2.4582, val loss 2.4890
step 6000: train loss 2.4520, val loss 2.4902
step 7000: train loss 2.4602, val loss 2.4921
step 8000: train loss 2.4526, val loss 2.4859
step 9000: train loss 2.4605, val loss 2.4944
step 10000: train loss 2.4618, val loss 2.4697
step 11000: train loss 2.4690, val loss 2.4919
step 12000: train loss 2.4511, val loss 2.4841
step 13000: train loss 2.4422, val loss 2.4964
step 14000: train loss 2.4541, val loss 2.4907
step 15000: train loss 2.4489, val loss 2.4872
step 16000: train loss 2.4612, val loss 2.4933
step 17000: train loss 2.4475, val loss 2.4943
step 18000: train loss 2.4470, val loss 2.4967
step 19000: train loss 2.4534, val loss 2.4989
step 20000: train loss 2.4415, val loss 2.4905
step 21000: train loss 2.4

In [76]:
print(decode(m.generate(torch.zeros(1,1, dtype=torch.long), max_new_tokens=500)[0].tolist()))



You:
I esspofouten bs'eernds Le thero t
IOfay ags Gubedret w u whap the ate moury ed foes com wsesounighallkimear, ft tw he n COLURonthin ws o avouser bushayour! siqur.
PO:
INI f
Why, atht touralf; lmband be-d ure anof tey feake blle aserdethe sthiffes wotes, NCEO:
ANThangetyst s.
Myo mu y tef hanhell,

Fotse s isthamod: RThang

Youth twimaco me heingoierar alf to d liompu rof rshe
Tho ndilur tupr y, unls comoull. m s thailld?
D er
LA h owave or t
SO:
Thitem yrldimy t ry,
Wit ll th t sunde.

h 
